In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.insert(0, "/SSF")
from DejaVu.explib.read_model import read_model

Using backend: pytorch


In [97]:
from pathlib import Path
exp_dir = Path('/data/SSF/experiment_outputs/run_GAT_node_classification.py.2021-12-22T03:50:07.949451')

from DejaVu.models import get_GAT_model
model, y_probs, y_preds = read_model(
    exp_dir,
    get_GAT_model,
    override_config=dict(data_dir=Path("/SSF/data/A1"), flush_dataset_cache=False)
)

cdp = fdg = model.fdg
config = model.config
cache = model.cache

output_dir = config.output_dir / 'gnnexplainer_explain'
output_dir.mkdir(exist_ok=True, parents=True)
print(output_dir)

2022-02-24 03:41:39.989 | DEBUG    | utils.load_model:best_checkpoint:19 - path=PosixPath('/data/SSF/experiment_outputs/run_GAT_node_classification.py.2021-12-22T03:50:07.949451') metric='val_loss' mode='min'
2022-02-24 03:41:40.014 | DEBUG    | utils.load_model:best_checkpoint:40 - ckpt_path=PosixPath('/data/SSF/experiment_outputs/run_GAT_node_classification.py.2021-12-22T03:50:07.949451/lightning_logs/version_0/checkpoints/epoch=449-A@1=0.647059-val_loss=0.296444-MAR=2.235294.ckpt') has metrics: {'epoch': '449', 'A@1': '0.647059', 'val_loss': '0.296444', 'MAR': '2.235294'}
2022-02-24 03:41:40.016 | DEBUG    | utils.load_model:best_checkpoint:40 - ckpt_path=PosixPath('/data/SSF/experiment_outputs/run_GAT_node_classification.py.2021-12-22T03:50:07.949451/lightning_logs/version_0/checkpoints/epoch=119-A@1=0.882353-val_loss=0.346040-MAR=1.470588.ckpt') has metrics: {'epoch': '119', 'A@1': '0.882353', 'val_loss': '0.346040', 'MAR': '1.470588'}
2022-02-24 03:41:40.017 | DEBUG    | utils.lo

/data/SSF/experiment_outputs/run_GAT_node_classification.py.2021-12-22T03:50:07.949451/gnnexplainer_explain


In [15]:
from DejaVu.models import DejaVuModelInterface, GAT
from typing import Tuple, List
import dgl
import torch as th
from DejaVu.dataset import DejaVuDataset
from einops import rearrange
import copy


class WrappedGATDejaVuModel(th.nn.Module):
    def __init__(self, GAT_aggregators, predictor):
        super().__init__()
        self.GAT_aggregators = copy.deepcopy(GAT_aggregators)
        self.predictor = copy.deepcopy(predictor)
        
    def forward(self, graph: dgl.DGLGraph, feat: th.Tensor, eweight=None):
        with graph.local_scope():
            for GAT_aggregator in self.GAT_aggregators:
                feat = rearrange(GAT_aggregator(graph, feat), "N H F -> N (H F)")

            node_weights = self.predictor(feat)
            return rearrange(node_weights, "N -> N 1")
        


def wrap_gnnexplainer_model_from_DejaVu_model(model: DejaVuModelInterface) -> Tuple[WrappedGATDejaVuModel, List[dgl.DGLGraph], List[th.Tensor]]:
    fdg = model.fdg
    module: GAT = model.module
    assert isinstance(module, GAT)
    feature_projector = module.feature_projector
    # shared_feature_mapper = module.shared_feature_mapper
    GAT_aggregators = module.GAT_aggregators
    predictor = module.predictor
    wrapped_module = WrappedGATDejaVuModel(GAT_aggregators, predictor)
    
    dataset = DejaVuDataset(
        cdp=model.fdg,
        feature_extractor=model.metric_preprocessor,
        fault_ids=fdg.failure_ids,
        window_size=model.config.window_size,
        augmentation=model.config.augmentation,
        normal_data_weight=1. if model.config.augmentation else 0.,
        drop_edges_fraction=model.config.drop_FDG_edges_fraction,
        device=model.device,
    )
    
    collate_fn = model.get_collate_fn(1)
    graph_list = []
    feat_list = []
    for fid in fdg.failure_ids:
        _features_list, _, _, _graph_list = collate_fn([dataset[fid]])
        
        _feat = feature_projector(_features_list)  # (batch_size, N, feature_size)

        # select the features of the instances that are retained in the graphs
        _feat = th.cat([_feat[i, g.ndata[dgl.NID]] for i, g in enumerate(_graph_list)])

        # _feat = (feature_mapper(_feat) + _feat) / 2
        
        graph_list.append(_graph_list[0])
        feat_list.append(_feat.detach())
    
    return wrapped_module, graph_list, feat_list
    
wrapped_module, graph_list, feat_list = wrap_gnnexplainer_model_from_DejaVu_model(model)

In [113]:
from graphviz import Digraph
import networkx as nx
failure_id=0

def plot_explain_graph(failure_id: int, num_hops=1) -> Digraph:
    from dgl.nn import GNNExplainer
    explainer = GNNExplainer(wrapped_module, num_hops=num_hops)

    rc = model.fdg.root_cause_instances_of(failure_id)[0]
    rc_gid = model.fdg.instance_to_gid(rc)
    print(f"{failure_id=} {model.fdg.root_cause_instances_of(failure_id)=}")
    new_center, sg, feat_mask, edge_mask = explainer.explain_node(
        rc_gid, 
        graph_list[failure_id], 
        feat_list[failure_id]
    )

    nid_dict = {i: nid.item() for i, nid in enumerate(sg.ndata[dgl.NID])}
    # print(nid_dict)
    for n in sg.nodes().tolist():
        n = model.fdg.gid_to_instance(nid_dict[n])

    nx_graph = nx.DiGraph()
    
    for u, v, weight in zip(*[_.tolist() for _ in sg.edges()], edge_mask):
        u = model.fdg.gid_to_instance(nid_dict[u])
        v = model.fdg.gid_to_instance(nid_dict[v])
        # print(u, v)
        if weight > 0.5 or u == rc or v == rc:
            color = f"gray{100 - int(weight * 100)}"
            nx_graph.add_edge(u, v, penwidth=f"{1 + weight}", color=color)
    nx_graph = nx_graph.subgraph([_ for _ in nx.weakly_connected_components(nx_graph) if rc in _][0])
            
    gv_graph = Digraph()
    for n in nx_graph.nodes():
        gv_graph.node(n, color="red" if n == rc else "cornflowerblue", style="filled", fontcolor="white")
    for u, v, data in nx_graph.edges(data=True):
        gv_graph.edge(u, v, **data)
    gv_graph.render(str(output_dir / f"{num_hops}" / f"{failure_id}_{rc}".replace(" ", "_")), format="pdf")
    return gv_graph

# plot_explain_graph(0, num_hops=5)
for num_hops in [1, 2, 3, 4, 5]:
    for fid in model.fdg.failure_ids:
        plot_explain_graph(fid, num_hops=num_hops)

failure_id=0 model.fdg.root_cause_instances_of(failure_id)=['docker_003 CPU']


Explain node {node_id}: 100%|██████████| 100/100 [00:03<00:00, 27.97it/s]


failure_id=1 model.fdg.root_cause_instances_of(failure_id)=['docker_002']


Explain node {node_id}: 100%|██████████| 100/100 [00:03<00:00, 25.14it/s]


failure_id=2 model.fdg.root_cause_instances_of(failure_id)=['docker_001']


...

Explain node {node_id}: 100%|██████████| 100/100 [00:03<00:00, 25.82it/s]


failure_id=76 model.fdg.root_cause_instances_of(failure_id)=['docker_006']


Explain node {node_id}: 100%|██████████| 100/100 [00:03<00:00, 25.25it/s]


failure_id=77 model.fdg.root_cause_instances_of(failure_id)=['docker_003']


Explain node {node_id}: 100%|██████████| 100/100 [00:04<00:00, 24.74it/s]


In [96]:
output_dir

PosixPath('/data/SSF/experiment_outputs/run_GAT_node_classification.py.2021-12-22T03:50:07.949451/LIME_explain')

In [28]:
sg.edges()

(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7,
         8, 8, 9, 9]),
 tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 5, 0, 6, 0, 7,
         0, 8, 0, 9]))

In [29]:
sg.ndata[dgl.NID]

tensor([ 1,  4,  7, 10, 13, 16, 18, 19, 20, 21])

In [39]:
zip(*[_.tolist() for _ in sg.edges()])